In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk

from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [12]:
# Набор данных взят с https://www.kaggle.com/aungpyaeap/fish-market
# Параметры нескольких популярных промысловых рыб
# length 1 = Body height
# length 2 = Total Length
# length 3 = Diagonal Length
fish_data = pd.read_csv("datasets/Fish.csv", delimiter=',')
print(fish_data)

    Species  Weight  Length1  Length2  Length3   Height   Width
0     Bream   242.0     23.2     25.4     30.0  11.5200  4.0200
1     Bream   290.0     24.0     26.3     31.2  12.4800  4.3056
2     Bream   340.0     23.9     26.5     31.1  12.3778  4.6961
3     Bream   363.0     26.3     29.0     33.5  12.7300  4.4555
4     Bream   430.0     26.5     29.0     34.0  12.4440  5.1340
..      ...     ...      ...      ...      ...      ...     ...
154   Smelt    12.2     11.5     12.2     13.4   2.0904  1.3936
155   Smelt    13.4     11.7     12.4     13.5   2.4300  1.2690
156   Smelt    12.2     12.1     13.0     13.8   2.2770  1.2558
157   Smelt    19.7     13.2     14.3     15.2   2.8728  2.0672
158   Smelt    19.9     13.8     15.0     16.2   2.9322  1.8792

[159 rows x 7 columns]


In [13]:
# Выделим входные параметры и целевое значение
x_labels = ['Height', 'Width']
y_label = 'Weight'

data = fish_data[x_labels + [y_label]]
print(data)

      Height   Width  Weight
0    11.5200  4.0200   242.0
1    12.4800  4.3056   290.0
2    12.3778  4.6961   340.0
3    12.7300  4.4555   363.0
4    12.4440  5.1340   430.0
..       ...     ...     ...
154   2.0904  1.3936    12.2
155   2.4300  1.2690    13.4
156   2.2770  1.2558    12.2
157   2.8728  2.0672    19.7
158   2.9322  1.8792    19.9

[159 rows x 3 columns]


In [14]:
# Определим размер валидационной и тестовой выборок
val_test_size = round(0.2*len(data))
print(val_test_size)

32


In [15]:
# Генерируем уникальный seed
my_code = "Соловьёв"
seed_limit = 2 ** 32
my_seed = int.from_bytes(my_code.encode(), "little") % seed_limit

In [16]:
# Создадим обучающую, валидационную и тестовую выборки
random_state = my_seed
train_val, test = train_test_split(data, test_size=val_test_size, random_state=random_state)
train, val = train_test_split(train_val, test_size=val_test_size, random_state=random_state)
print(len(train), len(val), len(test))

95 32 32


In [17]:
# Выделим обучающую, валидационную и тестовую выборки
train_x = train[x_labels]
train_y = np.array(train[y_label]).reshape(-1,1)

val_x = val[x_labels]
val_y = np.array(val[y_label]).reshape(-1,1)

test_x = test[x_labels]
test_y = np.array(test[y_label]).reshape(-1,1)

In [18]:
# Нормируем значения параметров
scaler_x = MinMaxScaler()
scaler_x.fit(train_x)
scaled_train_x = scaler_x.transform(train_x)

scaler_y = MinMaxScaler()
scaler_y.fit(train_y)
scaled_train_y = scaler_y.transform(train_y)

In [19]:
# Создадим модель метода k-ближайших соседей и обучим ее на нормированных данных. По умолчанию k = 5.
minmse=10
mink=0
a=[]
scaled_val_x = scaler_x.transform(val_x)
scaled_val_y = scaler_y.transform(val_y)
for k in range(1,51):
            model1 = KNeighborsRegressor(n_neighbors = k)
            model1.fit(scaled_train_x, scaled_train_y)
            val_predicted = model1.predict(scaled_val_x)
            mse1 = mean_squared_error(scaled_val_y, val_predicted)
            a.append(mse1)
            if mse1<minmse:
                minmse=mse1
                mink=k

In [20]:
print("Минимальная среднеквадратичная ошибка",minmse)
print("Значение k, соответствующее минимальной среднеквадратичной ошибке",mink)
print()
print(a)

Минимальная среднеквадратичная ошибка 0.005086151977539062
Значение k, соответствующее минимальной среднеквадратичной ошибке 4

[0.009907678344726561, 0.011437611755371092, 0.006591267157660589, 0.005086151977539062, 0.0072214669335937485, 0.007703570275200735, 0.0070445040283203105, 0.006191243652343749, 0.00615596741928289, 0.005492213369140625, 0.005420576347414127, 0.005501309004889594, 0.00607230342779216, 0.0063731620470942295, 0.006404322975260417, 0.007091906711578369, 0.006779372159858889, 0.006968559606481482, 0.007226147255683539, 0.007732193368225096, 0.007985668692037363, 0.008270548547665934, 0.008613314057146444, 0.008806245063993665, 0.008990540954101563, 0.00927004181540224, 0.009579155597786191, 0.00982058208185313, 0.01009415055391763, 0.01035960103027344, 0.010678332469864766, 0.01097702097427845, 0.011431939884574617, 0.011838955632404473, 0.012193577585399393, 0.012529042604140292, 0.012807524846346556, 0.01313160684330906, 0.013554288168441304, 0.0140425927972412

In [21]:
model1 = KNeighborsRegressor(n_neighbors = 4)
model1.fit(scaled_train_x, scaled_train_y)
val_predicted = model1.predict(scaled_val_x)
mse1 = mean_squared_error(scaled_val_y, val_predicted)
print(mse1)

0.005086151977539062


In [22]:
# Проверим результат на тестевойой выборке.
scaled_test_x = scaler_x.transform(test_x)
scaled_test_y = scaler_y.transform(test_y)

test_predicted = model1.predict(scaled_test_x)

mse2 = mean_squared_error(scaled_test_y,test_predicted)
print(mse2)

0.009878629226684572
